In [1]:
import findspark
findspark.init()
# Creating Spark Context
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext("local", "first app")
spark = SparkSession(sc)
import plotly.graph_objects as go
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [2]:
text_file = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true', quote='"', delimiter=',').load("C:\\Users\\mini_\\Desktop\Practica-Semi2\\archivos\\Covid19.csv")
rddfiltro = text_file.rdd.map(tuple)

In [3]:
#se toma el pais y sus numeros de casos
rddCOVID= rddfiltro.map(lambda word: (word[7],word[5]))
rddCOVID.take(10)

[('Afghanistan', 0),
 ('Afghanistan', 125),
 ('Afghanistan', 47),
 ('Afghanistan', 0),
 ('Afghanistan', 17),
 ('Afghanistan', 40),
 ('Afghanistan', 99),
 ('Afghanistan', 75),
 ('Afghanistan', 35),
 ('Afghanistan', 34)]

In [4]:
#se suman los numeros de casos reduciendo por la clave como pais
rddCONT=rddCOVID.reduceByKey(lambda a,b: a+b)
print("Conteo total -> %s" % rddCONT.collect())

Conteo total -> [('Afghanistan', 31391), ('Antigua_and_Barbuda', 1), ('Argentina', 622921), ('Armenia', 47431), ('Aruba', 3551), ('Australia', 26898), ('Austria', 38557), ('Azerbaijan', 39188), ('Bahamas', 3370), ('Bahrain', 64499), ('Bangladesh', 348916), ('Barbados', 189), ('Belarus', 75674), ('Belgium', 102201), ('Belize', 1627), ('Benin', 2280), ('Bermuda', 180), ('Bhutan', 261), ('Bolivia', 130676), ('Bonaire, Saint Eustatius and Saba', 36), ('Bosnia_and_Herzegovina', 25424), ('Botswana', 2567), ('Brazil', 4544629), ('British_Virgin_Islands', 71), ('Brunei_Darussalam', 145), ('Bulgaria', 18863), ('Burkina_Faso', 1846), ('Burundi', 473), ('Cambodia', 275), ('Cameroon', 20431), ('Canada', 143649), ('Cape_Verde', 5257), ('Cases_on_an_international_conveyance_Japan', 696), ('Cayman_Islands', 209), ('Central_African_Republic', 4793), ('Chad', 1151), ('Chile', 446274), ('China', 90369), ('Colombia', 765076), ('Comoros', 470), ('Congo', 4986), ('Costa_Rica', 63712), ('Cote_dIvoire', 1932

In [5]:
#se filtran los resultados
rddFINAL = rddCONT.filter(lambda x: "Cuba" in x[0] or "France" in x[0] or "Canada" in x[0] or "Singapore" in x[0] or "South_Korea" in x[0])
print("Resultados filtrados-> %s" % rddFINAL.collect())

Resultados filtrados-> [('Canada', 143649), ('Cuba', 5091), ('France', 453763), ('Singapore', 57576), ('South_Korea', 23045)]


In [6]:
#se orden los datos
rddFINAL = rddFINAL.sortBy(lambda x: x[1],False)

#se separan los nombre de los aeropuertos y totales
rddNombres = rddFINAL.map(lambda x: (x[0]))
rddTotales = rddFINAL.map(lambda x: (x[1]))
print(rddNombres.collect())
print(rddTotales.collect())

['France', 'Canada', 'Singapore', 'South_Korea', 'Cuba']
[453763, 143649, 57576, 23045, 5091]


In [7]:
#se realiza la gráfica de barras
fig = go.Figure(data=go.Bar(x=rddNombres.collect(),y=rddTotales.collect()))
fig.update_layout(title_text='Número total de casos de COVID-19',title_font_size=30,
                  yaxis=dict(title='Número de casos',title_font_size=25),
                  xaxis=dict(title='Nombre Pais',title_font_size=25))
fig.update_traces(overwrite=True, marker={"opacity": 0.5})
fig.write_html('R3_A1.html', auto_open=True)